# Tutorial 3: Dynamics of FM skyrmion

In this tutorial we will investigate the dynamics of an FM skyrmion driven by in plane spin current.
We will use the input for the FM skyrmion as in Example#1 in Tutorial#2, the LLG equation in this case reads as follows:

$$
\frac{\partial \mathbf{n}_i}{\partial t} =
\color{blue} {
- \frac{\gamma}{(1 + \alpha^2) \mu_i} \mathbf{n}_i \times \mathbf{B}_i^{\text{eff}}
- \frac{\gamma \alpha}{(1 + \alpha^2) \mu_i} \mathbf{n}_i \times (\mathbf{n}_i \times \mathbf{B}_i^{\text{eff}})
}
\color{red} {
- \frac{\alpha - \beta}{(1 + \alpha^2)} u \mathbf{n}_i \times (\hat{\mathbf{j}}_e \cdot \nabla_r) \mathbf{n}_i
+ \frac{1 + \beta \alpha}{(1 + \alpha^2)} u \mathbf{n}_i \times \left[\mathbf{n}_i \times (\hat{\mathbf{j}}_e \cdot \nabla_r) \mathbf{n}_i \right]
}
$$

Where the spin current strength related parameter is defined as $u$, injected at the direction of $\hat{\mathbf{j}}_e$

By applying in plane spin current along the x axis as shown below, the FM skyrmion wil encounter two forces the driving force and the Magnus force as shown below:

### FM skyrmion synamics under in-plane spin current

By applying an *in-plane spin current* applied along the *x-axis* on an *FM skyrmion*. 

As shown in the figure below, the skyrmion encounters two primary forces:  
- *The Driving Force* (propelling the skyrmion forward)  
- *The Magnus Force* (causing a transverse drift)  

This transverse motion leads to a *skyrmion Hall effect*, characterized by the **skyrmion Hall angle**—the deviation of the skyrmion's trajectory from straight-line motion.  
<p align="center">
    <img src="./assets/dynamics.png" alt="Dynamics of FM skyrmion" style="display: block; margin-left: auto; margin-right: auto; width:40%;">
</p>

---

### Simulating skyrmion motion

The **Dynamics.py** script applies an *in-plane spin current* to the skyrmion used in *Tutorial #2* and captures its motion. The script:  
1. Applies the spin current along the *x-axis*.  
2. Captures the skyrmion’s position after *every 1000 iterations*, which corresponds to *1 nanosecond*.  
3. Runs for a total of *5000 iterations*.  

---

### Extracting simulation data
To analyze the skyrmion’s motion, use the following command to inspect the relevant part of the output:

```bash
cat dynamics.txt
```

In [ ]:
from Dynamics import *

# Configuration file
cfgfile = "./input_FM_skyrmion.cfg"
n_iterations = 5000  # Total number of simulation steps
n_itertions_step = 1000  # Steps per iteration

total = []
quiet = True

with state.State(cfgfile, quiet) as p_state:
    io.image_read(p_state, "FM_skyrmion.ovf")
    system.update_data(p_state)
    alpha=0.2 #alpha value
    u=3 #current density related parameter
    parameters.llg.set_damping(p_state, alpha)#alpha value
    parameters.llg.set_stt(p_state, True, u, [1, 0, 0])

    simulation.start(p_state, simulation.METHOD_LLG, simulation.SOLVER_DEPONDT, single_shot=True, n_iterations=n_iterations)

    for x in range(int(n_iterations / n_itertions_step)):
        simulation.n_shot(p_state, n_itertions_step)

        if x == 0:
            io.image_write(p_state, "spins_0.ovf")  # Save the initial spin file

        if x == int(n_iterations / n_itertions_step) - 1:
            io.image_write(p_state, "spins_final.ovf")  # Save only the final spin file

        spins = np.array(system.get_spin_directions(p_state))
        sublattice(p_state, "spins.ovf", "positions.ovf")
        center = get_center(np.loadtxt("positions.ovf"), np.loadtxt("spins.ovf"))

        total.append([center[0], center[1]])

    simulation.stop_all(p_state)
